In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import os

notebook_path = os.path.abspath('')
import sys
# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'ensemble_commit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        print(commit_fit_path)
        break
 
if commit_fit_path is None:
    raise ValueError("Path containing 'ensemble_commit' not found in notebook path.")


if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

# import ensemble_model.preprocesser as preprocesser 
# import ensemble_model.MoE_model_CE_loss as moe 
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 

from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer, DistilBertModel


root
ensemble_commit
/root/ensemble_commit


In [2]:
df = pd.read_csv(r'../datasets/dataset.csv', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df

train, _ = train_test_split(df, train_size=0.35, random_state=42)
train, test = train_test_split(train, test_size=0.3, random_state=42)
test, val = train_test_split(test, train_size=0.5, random_state=42)

train.reset_index(inplace=True)
test.reset_index(inplace=True)
val.reset_index(inplace=True)

/tmp/ipykernel_2195/1430087705.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


In [3]:
# df['command'] = df['diff'].apply(lambda x : extract_lines(find_longest_list([y.changes for y in whatthepatch.parse_patch(x)])))
# extracted_cols = df['diff'].str.extract(r'^diff --git (?P<source>[^\t\n]+) (?P<target>[^\t\n]+)')
# whatthepatch.apply_diff(diff, lao)

In [4]:
# len(df_dataset)

In [5]:
# df_dataset[0]

In [6]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('../models/bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('../models/bert-base-cased')

codebert_model = RobertaModel.from_pretrained('../models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('../models/codebert-base')

Some weights of the model checkpoint at ../models/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
train

,index,Unnamed: 0,label,message,diff
0,897,897,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
1,2072,2072,0,allocator_thread: rework message sending struc...,diff --git a/src/allocator_thread.c b/src/allo...
2,8489,8489,1,...,diff --git a/coders/mpc.c b/coders/mpc.c\ninde...
3,1692,1692,0,Fix phpcs,diff --git a/src/Core/Configuration.php b/src/...
4,7905,7905,0,Merge pull request #6467 from BrickOzp/attach2...,diff --git a/Themes/default/Display.template.p...
...,...,...,...,...,...
2460,477,477,0,handles FNC1,diff --git a/lib/barby/barcode/code_128.rb b/l...
2461,7035,7035,0,Bump tox from 3.20.1 to 3.21.3\n\nBumps [tox](...,diff --git a/requirements/dev.txt b/requiremen...
2462,3539,3539,1,"Strengthen sanitization, fixes #817",diff --git a/geo-mashup-db.php b/geo-mashup-db...
2463,5333,5333,0,modify short open tag <? to <?php\n\nThe short...,diff --git a/cookieviz/settings.inc b/cookievi...


In [8]:
# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)

In [9]:
# Create stacking model
combined_model = cm.CombinedModel(base_model1, base_model2)
# Train the model
combined_model.trainer(train_loader, val_loader,num_epochs=10, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10 Loss: 0.4513: 100%|██████████| 309/309 [03:08<00:00,  1.64batch/s]


=============================train========================


Epoch 2/10 Loss: 0.2140: 100%|██████████| 309/309 [02:54<00:00,  1.77batch/s]


=============================train========================


Epoch 3/10 Loss: 0.0944: 100%|██████████| 309/309 [02:52<00:00,  1.79batch/s]


...

=============================train========================


Epoch 8/10 Loss: 0.0327: 100%|██████████| 309/309 [02:52<00:00,  1.79batch/s]


=============================train========================


Epoch 9/10 Loss: 0.0232: 100%|██████████| 309/309 [02:55<00:00,  1.76batch/s]


=============================train========================
Early stopping triggered after 9 epochs
Validation Accuracy: 0.8526
Precision: 0.8586
Recall: 0.8526
F1-Score: 0.8535
              precision    recall  f1-score   support

           0     0.9091    0.8333    0.8696       312
           1     0.7860    0.8802    0.8304       217

    accuracy                         0.8526       529
   macro avg     0.8475    0.8568    0.8500       529
weighted avg     0.8586    0.8526    0.8535       529



In [10]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = combined_model.evaluate(test_loader)

Validation Accuracy: 0.8788
Precision: 0.8829
Recall: 0.8788
F1-Score: 0.8798
              precision    recall  f1-score   support

           0     0.9274    0.8776    0.9018       335
           1     0.8057    0.8808    0.8416       193

    accuracy                         0.8788       528
   macro avg     0.8666    0.8792    0.8717       528
weighted avg     0.8829    0.8788    0.8798       528

